In [191]:
import models
import dataset
import tensorflow as tf
from tensorflow.keras.layers import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
import cv2.cv2 as cv2

In [192]:
def get_data(feature_dir, label_dir):
    features, _ = dataset.load_face_pictures(feature_dir, img_num=64, color_mode="rgb")
    labels, _ = dataset.load_face_pictures(label_dir, img_num=64, color_mode="grayscale")
    features = features / 255
    labels = labels / 255

    return features, labels

In [ ]:
#!g1.1
if __name__ == '__main__':
    input_layer = Input((256, 256, 3))
    model = models.standard_unet(input_layer, 16)

    model.compile(Adam(), loss='binary_crossentropy', metrics=["accuracy"])

    epochs = 20000
    for epoch in range(epochs):
        features, labels = get_data("train_data/medical/CelebA-HQ-img-256-256-masked", "train_data/medical/CelebA-HQ-img-256-256-labels")
        loss_acc = model.train_on_batch(features, labels)
        print("[Epoch %d/%d] [D loss_whole: %f, acc_iou: %f]" % (epoch, epochs, loss_acc[0], loss_acc[1]))
        if epoch % 100 == 0:
            features, labels = get_data(
                "train_data/medical/CelebA-HQ-img-256-256-masked",
                "train_data/medical/CelebA-HQ-img-256-256-labels")
            predictions = model.predict(features)
            cv2.imwrite("gan_images/" + str(epoch)+ "_segment.png", (predictions[0] * 255).astype('uint8'))
            cv2.imwrite("gan_images/" + str(epoch)+ "_segment_real.png", (features[0] * 255).astype('uint8'))
            model.save("saved_models/" + str(epoch) + "segment_net", save_format="tf")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Traceback (most recent call last):
  File "/kernel/lib/python3.7/site-packages/ml_kernel/state/state.py", line 96, in __eq__
    self._unwrap()
  File "/kernel/lib/python3.7/site-packages/ml_kernel/state/state.py", line 79, in _unwrap
    self._self_state.load_lazy(self.varname)
  File "/kernel/lib/python3.7/site-packages/ml_kernel/state/state.py", line 331, in <lambda>
    self._ml_state.load_lazy = lambda var: load_lazy(self, var, branch)
  File "/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py", line 292, in _load_lazy
    with state_manager.state.loading_state:
  File "/usr/lib/python3.7/contextlib.py", line 112, in __enter__
    return next(self.gen)
  File "/kernel/lib/python3.7/site-packages/ml_kernel/state/state.py", line 185, in loading_state
    raise Exception("already loading state")
Exception: already loading state
Traceback (most recent call last):
  File "/kernel/lib/python3.7/site-packages/ml_kernel/state/state.py", line 96, in __eq__
    self._unwrap()
  File "/

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Traceback (most recent call last):
  File "/kernel/lib/python3.7/site-packages/ml_kernel/state/state.py", line 96, in __eq__
    self._unwrap()
  File "/kernel/lib/python3.7/site-packages/ml_kernel/state/state.py", line 79, in _unwrap
    self._self_state.load_lazy(self.varname)
  File "/kernel/lib/python3.7/site-packages/ml_kernel/state/state.py", line 331, in <lambda>
    self._ml_state.load_lazy = lambda var: load_lazy(self, var, branch)
  File "/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py", line 292, in _load_lazy
    with state_manager.state.loading_state:
  File "/usr/lib/python3.7/contextlib.py", line 112, in __enter__
    return next(self.gen)
  File "/kernel/lib/python3.7/site-packages/ml_kernel/state/state.py", line 185, in loading_state
    raise Exception("already loading state")
Exception: already loading state
Traceback (most recent call last):
  File "/kernel/lib/python3.7/site-packages/ml_kernel/state/state.py", line 96, in __eq__
    self._unwrap()
  File "/

In [23]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.utils import *
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import *
from tensorflow.keras.datasets import *
from sklearn.model_selection import train_test_split
import PIL.Image as Image
import dataset
import models
import tensorflow as tf

if __name__ == '__main__':

    dir = "medical/CelebA-HQ-img-256-256-masked"
    features = dataset.load_face_pictures(dir, img_num=100, color_mode='rgb') / 255

    model = load_model("saved_models/unet_segment.h5", compile=False)
    model.summary()
    predictions = model.predict(features)[:,:,:,0]

    for i in range(len(predictions)):
        predicted_img = Image.fromarray((predictions[i] * 255.0).astype("uint8"), "L")
        # real_label = Image.fromarray((l_test[i] * 255).astype("uint8"), "RGB")
        predicted_img.save("results/" + str(i) + ".png")
        # real_label.save("results/real_" + str(i) + ".png")

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 16) 2320        conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 128, 128, 16) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:882: UserWarning: The following variables cannot be serialized: model
  warnings.warn(message)


In [39]:
%pip install 'tensorflow==1.15.0' --force-reinstall

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-1.15.0-cp37-cp37m-manylinux2010_x86_64.whl (412.3 MB)
  Using cached absl_py-0.12.0-py3-none-any.whl (129 kB)
  Using cached grpcio-1.36.1-cp37-cp37m-manylinux2014_x86_64.whl (4.1 MB)
  Using cached protobuf-3.15.6-cp37-cp37m-manylinux1_x86_64.whl (1.0 MB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached Keras_Applications-1.0.8-py3-none-any.whl (50 kB)
  Using cached numpy-1.20.2-cp37-cp37m-manylinux2010_x86_64.whl (15.3 MB)
  Using cached six-1.15.0-py2.py3-none-any.whl (10 kB)
  Using cached gast-0.2.2-py3-none-any.whl
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached astor-0.8.1-py2.py3-none-any.whl (27 kB)
  Using cached tensorflow_estimator-1.15.1-py2.py3-none-any.whl (503 kB)
  Using cached wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl
  Using cached termcolor-1.1.0-